In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# ---------------Generating URL for product--------------

def get_url(product):
    product = product.replace(' ','+')
    template = 'https://www.lazada.com.my/catalog/?q={}'
    url = template.format(product)
    return url

# ---------------Generating Info for product--------------

def get_all_products(card):
    pImg = card.find('img','jBwCF')
    product_image = pImg['src']
    product_name = card.find('div','RfADt').text.strip()
    #This to ignore special characters/emoji --> change into bytecode
    product_name = product_name.encode('ascii','ignore')
    #This to remove bytecode(B) in product_name
    product_name = str(product_name,'utf-8')
    product_price = card.find('span','ooOxS').text.strip()
    anchor_tag = card.a.get('href')
    product_buy_link = anchor_tag.lstrip("/")
    
    product_info = (product_image,product_name,product_price,product_buy_link)
    
    return product_info

#---------------Main Function--------------

def main(product):
    records = []
    url = get_url(product)
    
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    #Define an initial value
    temp_height=0
    while True:
        #Looping down the scroll bar
        driver.execute_script("window.scrollBy(0,1000)")
        #sleep and let the scroll bar react
        time.sleep(5)
        #Get the distance of the current scroll bar from the top
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
        #If the two are equal to the end
        if check_height==temp_height:
            break
        temp_height=check_height
    
    time.sleep(10)
    
    #Creating a BSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    product_cards = soup.find_all('div','Bm3ON')
    
    for everyCard in product_cards:
        productDetails = get_all_products(everyCard)
        records.append(productDetails)
        
    # -------Using Pandas to Save all info-------
    
    col = ['product_image','product_name','product_price','product_buy_link']
    
    lazada_data = pd.DataFrame(records,columns=col)
    
    lazada_data.to_csv('C:\\Users\\user\\Desktop\\web scraping task\\lazadaData1.csv',encoding='utf-8')

In [2]:
#----Receiving user input-----

product = input('What do you want to buy? : ')
main(product)

What do you want to buy? : jeans


<ipython-input-1-22d8e3822c1b>:41: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe')
